# Data Preprocessing for Online Retail Stores

The data is acquired from [Kaggle](https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci). It contains transactions of a an online retail transactions for a UK-based and registered, online retail between 2009-12-01 and 2011-12-09. The company mainly sells unique all-occasion gift-ware.


In [ ]:
import numpy as np
import pandas as pd
import os

pd.options.display.max_columns = 999
pd.options.display.float_format = "{:.2f}".format

I have put my dataset in a google drive folder and I need to mount the google drive to access it.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

data_path = '/content/drive/My Drive/databank/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Read Data

Read the dataset from csv file.

In [ ]:
df_order = pd.read_csv(data_path + 'online_retail_store.csv')

df_order.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.00,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.00,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.00,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.00,United Kingdom


**Data Description**:

- Invoice: the invoice id of the transaction
- StockCode: sku or id of item purchased
- Description: Name of the sku
- InvoiceDate: timestamp of the purchase
- Price: price per unit of item
- Customer ID: Id of the customer
- Country: The name of the country where a customer resides

## Data Preprocessing

First, I want to tidy up the column name by transforming it into snake case and rename the invoice date into invoice time.

In [ ]:
# lowercase column name
df_order.columns = df_order.columns.str.lower()
df_order.columns = df_order.columns.str.replace(' ', '_')

# rename column
df_order.rename({'invoicedate':'invoice_time'}, axis=1, inplace = True)

df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   invoice       1067371 non-null  object 
 1   stockcode     1067371 non-null  object 
 2   description   1062989 non-null  object 
 3   quantity      1067371 non-null  int64  
 4   invoice_time  1067371 non-null  object 
 5   price         1067371 non-null  float64
 6   customer_id   824364 non-null   float64
 7   country       1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


### Change Data Type

The following are data type and new column creation for the transactions:

- change invoice_time object to timestamp
- change customer_id from float to string
- create new column invoice_date that only contains the date and no time
- create sales column from quantity * price
- uppercase and remove leading and trailing whitespace of all ID to prevent duplicate entries or unmatched data


In [ ]:
# change data type
df_order['invoice_time'] = pd.to_datetime(df_order['invoice_time'])
df_order['customer_id'] = df_order['customer_id'].astype('Int64', errors = 'ignore').astype('str', errors = 'ignore').replace('<NA>', None)

# new column
df_order['invoice_date'] = df_order['invoice_time'].dt.to_period('D').dt.to_timestamp()
df_order['sales']  = df_order['quantity'] * df_order['price']

# uppercase
df_order['stockcode'] = df_order['stockcode'].str.upper().str.strip()
df_order['customer_id'] = df_order['customer_id'].str.upper().str.strip()
df_order['invoice'] = df_order['invoice'].str.upper().str.strip()

df_order.head()

,invoice,stockcode,description,quantity,invoice_time,price,customer_id,country,invoice_date,sales
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,2009-12-01,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,2009-12-01,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,2009-12-01,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,2009-12-01,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,2009-12-01,30.00


### Inspect Duplicated Data

Let's check if there is any duplicated data.

In [ ]:
df_order.duplicated().sum()

34335

In [ ]:
df_order[ df_order.duplicated() ][['invoice', 'stockcode']].merge(df_order, on = ['invoice', 'stockcode']).head()

,invoice,stockcode,description,quantity,invoice_time,price,customer_id,country,invoice_date,sales
0,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329,United Kingdom,2009-12-01,3.75
1,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329,United Kingdom,2009-12-01,3.75
2,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329,United Kingdom,2009-12-01,3.75
3,489517,22130,PARTY CONE CHRISTMAS DECORATION,6,2009-12-01 11:34:00,0.85,16329,United Kingdom,2009-12-01,5.10
4,489517,22130,PARTY CONE CHRISTMAS DECORATION,6,2009-12-01 11:34:00,0.85,16329,United Kingdom,2009-12-01,5.10


It look like the data has duplicate entry, since all of the column has identical values. Sometimes a single invoice can have identical stockcode entered multiple times, e.g. different discount or different batch.

Since there is no further information that can be acquired regarding this manner, we will drop this data duplicate.

In [ ]:
df_order.drop_duplicates(inplace = True)

### Inspect Missing Values

Let's continue by checking the missing values. There are 2 columns that has missing values: customer ID and item description.

In [ ]:
# Check missing values
df_order.isnull().sum()

,0
invoice,0
stockcode,0
description,4275
quantity,0
invoice_time,0
price,0
customer_id,235151
country,0
invoice_date,0
sales,0


#### Customer ID

Let's sample the data for transaction with NULL customer ID.

In [ ]:
df_order[ df_order['customer_id'].isnull() ].head(10)

,invoice,stockcode,description,quantity,invoice_time,price,customer_id,country,invoice_date,sales
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,None,United Kingdom,2009-12-01,-0.00
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,None,United Kingdom,2009-12-01,-0.00
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,None,United Kingdom,2009-12-01,-0.00
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,None,United Kingdom,2009-12-01,-0.00
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,None,United Kingdom,2009-12-01,0.55
578,489525,85227,SET/6 3D KIT CARDS FOR KIDS,1,2009-12-01 11:49:00,0.85,None,United Kingdom,2009-12-01,0.85
1055,489548,22271,FELTCRAFT DOLL ROSIE,1,2009-12-01 12:32:00,2.95,None,United Kingdom,2009-12-01,2.95
1056,489548,22254,FELT TOADSTOOL LARGE,12,2009-12-01 12:32:00,1.25,None,United Kingdom,2009-12-01,15.00
1057,489548,22273,FELTCRAFT DOLL MOLLY,3,2009-12-01 12:32:00,2.95,None,United Kingdom,2009-12-01,8.85
1058,489548,22195,LARGE HEART MEASURING SPOONS,1,2009-12-01 12:32:00,1.65,None,United Kingdom,2009-12-01,1.65


Looks like a transaction can have NULL customer ID, since it present in a successfull and cancelled transaction. We will assume that the NULL customer ID refers to guest transaction or a transaction where the user doesn't login with email or other credential.

In [ ]:
# classify as guest and member
df_order['flag_customer'] = np.where(df_order['customer_id'].isnull(), 'Guest', 'Member')

df_order.value_counts('flag_customer')

,count
flag_customer,
Member,797885
Guest,235151


#### Item Description

Let's check the blank item description.

In [ ]:
df_order[ df_order['description'].isnull() ].head()

,invoice,stockcode,description,quantity,invoice_time,price,customer_id,country,invoice_date,sales,flag_customer
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,None,United Kingdom,2009-12-01,-0.00,Guest
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.00,None,United Kingdom,2009-12-01,-0.00,Guest
3161,489659,21350,NaN,230,2009-12-01 17:39:00,0.00,None,United Kingdom,2009-12-01,0.00,Guest
3731,489781,84292,NaN,17,2009-12-02 11:45:00,0.00,None,United Kingdom,2009-12-02,0.00,Guest
4296,489806,18010,NaN,-770,2009-12-02 12:42:00,0.00,None,United Kingdom,2009-12-02,-0.00,Guest


The strategy to fill the blank item name is by checking at other transaction with the same stock code. But first, let's check if a stock code can have duplicate name.

In [ ]:
# named sku
all_sku_name = df_order[ ~df_order['description'].isnull() ][['stockcode', 'description']].drop_duplicates()

all_sku_name['stockcode'].duplicated().sum()

1593

In [ ]:
all_sku_name[ all_sku_name['stockcode'].duplicated() ].head()

,stockcode,description
171,22195,LARGE HEART MEASURING SPOONS
263,21733,85123a mixed
284,85123A,21733 mixed
332,71477,COLOUR GLASS. STAR T-LIGHT HOLDER
607,22232,JIGSAW TOADSTOOLS 3 PIECE


Same stock code can have different name. To mitigate this, let's just take the latest item description only. To make sure we get the correct name, we will exclude cancelled transaction when checking the latest name.

In [ ]:
# sort the transaction data
df_order = df_order.sort_values('invoice_time', ascending = False)

# get latest sku name
latest_sku_name = df_order[ (~df_order['description'].isnull()) & (df_order['sales'] > 0) ][['stockcode', 'description']].drop_duplicates(subset = 'stockcode', keep='first')

Let's drop the initial item description column and use the latest name.

In [ ]:
df_order = df_order.drop(columns = 'description').merge(latest_sku_name, on = 'stockcode', how = 'left')

df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033036 entries, 0 to 1033035
Data columns (total 11 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   invoice        1033036 non-null  object        
 1   stockcode      1033036 non-null  object        
 2   quantity       1033036 non-null  int64         
 3   invoice_time   1033036 non-null  datetime64[ns]
 4   price          1033036 non-null  float64       
 5   customer_id    797885 non-null   object        
 6   country        1033036 non-null  object        
 7   invoice_date   1033036 non-null  datetime64[ns]
 8   sales          1033036 non-null  float64       
 9   flag_customer  1033036 non-null  object        
 10  description    1032599 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(6)
memory usage: 86.7+ MB


Some stock code still has no description, but since the number is small, we will leave it as it is and will not be excluded from reporting.

### Inspect Negative Sales

Negative sales or cancelled transaction is a common thing in an order dataset. We will keep this data since removing it will distort the reporting and may not reflect the actual revenue generated by the company.

In [ ]:
df_order[ df_order['sales'] < 0 ].head()

,invoice,stockcode,quantity,invoice_time,price,customer_id,country,invoice_date,sales,flag_customer,description
192,C581569,20979,-5,2011-12-09 11:58:00,1.25,17315,United Kingdom,2011-12-09,-6.25,Member,36 PENCILS TUBE RED RETROSPOT
193,C581569,84978,-1,2011-12-09 11:58:00,1.25,17315,United Kingdom,2011-12-09,-1.25,Member,HANGING HEART JAR T-LIGHT HOLDER
194,C581568,21258,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom,2011-12-09,-54.75,Member,VICTORIAN SEWING BOX LARGE
361,C581499,M,-1,2011-12-09 10:28:00,224.69,15498,United Kingdom,2011-12-09,-224.69,Member,Manual
1453,C581490,23144,-11,2011-12-09 09:57:00,0.83,14397,United Kingdom,2011-12-09,-9.13,Member,ZINC T-LIGHT HOLDER STARS SMALL


## Save Data

In [ ]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033036 entries, 0 to 1033035
Data columns (total 11 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   invoice        1033036 non-null  object        
 1   stockcode      1033036 non-null  object        
 2   quantity       1033036 non-null  int64         
 3   invoice_time   1033036 non-null  datetime64[ns]
 4   price          1033036 non-null  float64       
 5   customer_id    797885 non-null   object        
 6   country        1033036 non-null  object        
 7   invoice_date   1033036 non-null  datetime64[ns]
 8   sales          1033036 non-null  float64       
 9   flag_customer  1033036 non-null  object        
 10  description    1032599 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(6)
memory usage: 86.7+ MB


In [ ]:
# master customer
master_customer = df_order[ ~df_order['customer_id'].isnull() ][['customer_id', 'country']].drop_duplicates()

# master item
master_item = df_order[ ~df_order['description'].isnull() ][['stockcode', 'description']].drop_duplicates()

Let's check if one customer_id can only have one country

In [ ]:
master_customer[ master_customer.duplicated(subset = 'customer_id') ].head()

,customer_id,country
151123,12431,Belgium
163573,12457,Cyprus
299314,12422,Switzerland
360261,12394,Belgium
366892,12417,Spain


In [ ]:
master_customer[ master_customer['customer_id'] == '12431']

,customer_id,country
100983,12431,Australia
151123,12431,Belgium


Turns out one customer_id can have multiple country, so we will not save the country information separately from the order dataset. We will remove the description from the order dataset.

In [ ]:
# orders
df_order_final = df_order.drop(['description'], axis = 1).copy()

df_order_final.head()

,invoice,stockcode,quantity,invoice_time,price,customer_id,country,invoice_date,sales,flag_customer
0,581587,POST,1,2011-12-09 12:50:00,18.00,12680,France,2011-12-09,18.00,Member
1,581587,22367,8,2011-12-09 12:50:00,1.95,12680,France,2011-12-09,15.60,Member
2,581587,22631,12,2011-12-09 12:50:00,1.95,12680,France,2011-12-09,23.40,Member
3,581587,22555,12,2011-12-09 12:50:00,1.65,12680,France,2011-12-09,19.80,Member
4,581587,22728,4,2011-12-09 12:50:00,3.75,12680,France,2011-12-09,15.00,Member


In [ ]:
# save the data
df_order_final.to_csv(data_path + 'order_online_retail.csv', index = False)

master_item.to_csv(data_path + 'item_online_retail.csv', index = False)